# Vorlesung 12-1

In [1]:
using Interact
using Plots
using LaTeXStrings
using LinearAlgebra
using Printf
using Statistics

WebIO._IJuliaInit()

## Lagrange'sche Form der Polynominterpolation

mit zweiter baryzentrischer Formel

In [2]:
mutable struct Lagrange{T<:AbstractFloat}
    n             # order of polynomial
    x::Vector{T}  # nodes 
    λ::Vector{T}  # weights for barycentric formula
    f::Vector{T}  # f-values
    
    function Lagrange(x::Vector{T}) where T<:AbstractFloat
    # construction of weights
        n, λ = length(x), ones(T, size(x))
        for k = 1:n
            d = x[1:k-1] .- x[k]
            λ[1:k-1] ./=  d
            λ[k] /= prod(-d)
        end
        return new{T}(n, x, λ, zeros(T, size(x)))
    end
end

function LagrangeEval(p::Lagrange{T}, x::AbstractArray{T}) where {T<:AbstractFloat}
    num = den = zeros(T, size(x))
    for j=1:p.n                                         # 2nd barycentric formula
        num += (c = p.λ[j]./(x .- p.x[j]))*p.f[j]
        den += c
    end
    y = num./den
    ind = isnan.(y); y[ind] = p.f[indexin(x, p.x)[ind]] # evaluation at the nodes themselves
    return y
end

setf!(p::Lagrange{T}, f::Vector{T}) where {T<:AbstractFloat} = (p.f = f; p)

setf!(p::Lagrange{T}, f::Function)  where {T<:AbstractFloat} = setf!(p,f.(p.x))

Lagrange(x::Vector{T}, f) where {T<:AbstractFloat} = setf!(Lagrange(x), f)

LagrangeEval(p::Lagrange{T}, x::T) where {T<:AbstractFloat} = LagrangeEval(p, [x])[1]

(p::Lagrange)(x) = LagrangeEval(p, x) # enables calls of the form p(x)

## Kubische Spline-Interpolation

mit Not-A-Knot-Randbedingungen, vgl. Aufgabe 41

In [3]:
struct CubicSpline{T<:AbstractFloat} 
    n::Integer        # number of subintervals
    x::Vector{T}      # nodes
    f::Vector{T}      # values
    m::Vector{T}      # 2nd derivatives of the spline at nodes
    coeff::Array{T,2} # coefficients of local cubic Hermite representation
    
    function CubicSpline(x::Vector{T},f::Vector{T}) where T<:AbstractFloat 
    # construction of cubic spline
        if !issorted(x); error("nodes must be sorted"); end
        h = diff(x) 
        n = length(h)
        q = h[1:n-1]./(h[1:n-1]+h[2:n])
        p = h[2:n]./(h[1:n-1]+h[2:n])
        A = Tridiagonal([q[2:n-2];1-h[n]/h[n-1]],[2+h[1]/h[2];2*ones(n-3);2+h[n]/h[n-1]],[1-h[1]/h[2];p[2:n-2]])
        df = diff(f)./h
        ddf = diff(df)./(x[3:n+1]-x[1:n-1])
        m = lu(A, Val(false))\ddf # no pivoting
        m = [(m[1]-q[1]m[2])/p[1]; m; (m[n-1]-p[n-1]m[n-2])/q[n-1]]
        coeff = [f[1:n] df (2*m[1:n]+m[2:n+1]) diff(m)./h]
        return new{T}(n, x, f, m, coeff)
    end
end

function splineval(s::CubicSpline{T}, x::T) where {T<:AbstractFloat} # evaluation of cubic spline
    n = s.n
    j = try findall(s.x[1:n] .<= x .<= s.x[2:n+1])[1] catch e throw(DomainError) end
    return ((s.coeff[j,4]*(x-s.x[j])+s.coeff[j,3])*(x-s.x[j+1])+s.coeff[j,2])*(x-s.x[j])+s.coeff[j,1]
end

CubicSpline(x::Vector{T}, f::Function) where {T<:AbstractFloat} = CubicSpline(x, f.(x))

(s::CubicSpline)(x::Real) = splineval(s,x) # enables calls of the form s(x)
(s::CubicSpline)(x::AbstractArray) = s.(x) # enables vectorization (pointwise evaluation)

### Zwei Standard-Familien von Stützstellen

In [4]:
ChebyshevKnots(a, b, n) = (a+b)/2 .+ (b-a)/2*cos.(range(0,π,length=n+1))
EquidistantKnots(a, b, n) = range(a, b, length=n+1) |> collect;

## Vergleich der kubischen Splineinterpolation mit der Polynom-Interpolation

### Runge-Funktion

In [5]:
α = 5
f(x) = 1 ./(1+α^2*x.^2)

# Konvergenzkonstante im Satz von Bernstein

ρ = √(1+1/α^2)+1/α

# Punkte zum Auswerten von Fehlern und für die Plots

xx = range(-1, 1, length=2000);

### Interaktiver Plot

In [6]:
@manipulate for n=8:52, method = Dict("Spline" => CubicSpline, "Lagrange" => Lagrange), 
    knots = Dict("equidistant" => EquidistantKnots, "Chebyshev" => ChebyshevKnots)
    x = sort(knots(-1,1,n))
    y = f.(x)
    yy = method(x,y)(xx)
    plot(xx, yy, ylims = (-0.2, 1.2), linewidth = 2, title = @sprintf("\n%s-Interpolation\n",method))
    scatter!(x, y, xlabel = L"$x$", ylabel = L"$p(x)$", legend = false)
    # title(@sprintf("%s-Interpolation",method{eltype(x)}))
    # xticks(linspace(-1,1,9))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 52, :min => 8, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
30, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001507a2a90, Task (runnable) @0x00000001507a2a90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40\",\"41\",\"42\",\"43\",\"44\",\"45\",\

## Illustration der Konvergenzsätze

In [7]:
# exponentielle Konvergenz für Polynominterpolation

err_exponential(t,c) = c*ρ.^(-t)

# algebraische Konvergenz für Spline-Interpolation

q = 4.0; # not-a-knot cubic spline has order 4
err_algebraic(t,c) = c*t.^(-q);

In [8]:
@manipulate for method = Dict("Lagrange" => Lagrange, "Spline" => CubicSpline), 
        scaling = Dict("semilogy" => :linear, "loglog" => :log)  
    # Vorbereitung
    if method==CubicSpline
        rg = Int.(round.(2 .^range(5,10, length=20)))
        ylabelstr = L"interpolation error $E_n = ||f-s_n||_\infty$"
        xlabelstr = L"number of subintervals $n$"
        err = err_algebraic
    else
        rg, fit = 10:4:200, 1:20
        ylabelstr = L"interpolation error $E_n = ||f-p_n||_\infty$"
        xlabelstr = L"polynomial degree $n$"
        err = err_exponential
    end
    
    # Bestimmung des Interpolationsfehlers
    E = zeros(maximum(rg))
    for n in rg
        p = method(sort(ChebyshevKnots(-1,1,n)),f)
        E[n] = norm(f.(xx).-p(xx),Inf)
    end
    
    # Fit der theoretischen Vorhersage
    c = (method==CubicSpline) ? mean(E[rg].*(rg.^q)) : mean(E[rg[fit]].*(ρ.^rg[fit]))
    
    # Plot
    plot(rg, err(rg,c), yaxis=:log, linewidth = 2, label = "theory")
    (scaling == :log) && plot!(xaxis = :log)
    scatter!(rg,E[rg], ylims = (1e-15,1e0), title = "\nKonvergenzplot\n", label = @sprintf("%s",method))
    scatter!(ylabel = ylabelstr, xlabel = xlabelstr)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["method"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Vector{OrderedDict}} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "Lagrange", "val" => 1, "id" => "idtmaMgAkD"), OrderedDict{String, Any}("key" => "Spline", "val" => 2, "id" => "idU66kmVjN")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/Folkmar/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/Folkmar/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000150caefe0, Task (runnable) @0x0000000150caefe0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var 